In [17]:
import momi
import os
import logging
import argparse
logging.basicConfig(level=logging.INFO,
                    filename="tutorial.log")
os.chdir('/Users/chichun/Desktop/workspace/AdmixtreGraph2020')

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--replicate', '-r', type=str, nargs='+', help='replicate')
args = my_parser.parse_args()
replicate_id = args.replicate

In [13]:
# simulating Data
recoms_per_gen = 1.25e-8
bases_per_locus = int(5e6)
n_loci = 100
ploidy = 2  

# n_alleles per population (n_individuals = n_alleles / ploidy)
sampled_n_dict = {"Outgroup": 4,"S1":8, "S2":8, "Adm":8}


In [14]:
# a dict mapping samples to populations
ind2pop = {}
for pop, n in sampled_n_dict.items():
    for i in range(int(n / ploidy)):
        # in the vcf, samples are named like YRI_0, YRI_1, CHB_0, etc
        ind2pop["{}_{}".format(pop, i)] = pop

with open("data/ind2pop.txt", "w") as f:
    for i, p in ind2pop.items():
        print(i, p, sep="\t", file=f)

!cat data/ind2pop.txt

Outgroup_0	Outgroup
Outgroup_1	Outgroup
S1_0	S1
S1_1	S1
S1_2	S1
S1_3	S1
S2_0	S2
S2_1	S2
S2_2	S2
S2_3	S2
Adm_0	Adm
Adm_1	Adm
Adm_2	Adm
Adm_3	Adm


In [21]:
replicate_id = "1"
bashCommand = "python -m momi.read_vcf data/{0}.vcf.gz data/ind2pop.txt data/{0}.snpAlleleCounts.gz --bed data/{0}.bed".format(replicate_id)
os.system(bashCommand)

0

In [23]:
bashCommand = "python -m momi.extract_sfs data/sfs_{0}.gz 100 data/{0}.snpAlleleCounts.gz".format(replicate_id)
os.system(bashCommand)

0

In [24]:
sfs = momi.Sfs.load("data/sfs_{}.gz".format(replicate_id))

In [25]:
no_pulse_model = momi.DemographicModel(N_e=5e4, gen_time=29, muts_per_gen=1.25e-8)
no_pulse_model.set_data(sfs)
# random initial value; user-specified lower bound
no_pulse_model.add_time_param("t_Adm_S2", lower=1e4)
no_pulse_model.add_leaf("Adm")
no_pulse_model.add_leaf("S2")
no_pulse_model.move_lineages("Adm", "S2", t="t_Adm_S2")
no_pulse_model.add_leaf("S1")
no_pulse_model.add_time_param("t_anc", lower=5e4, lower_constraints=["t_Adm_S2"])
no_pulse_model.move_lineages("S2", "S1", t="t_anc")
no_pulse_model.add_leaf("Outgroup")
no_pulse_model.add_time_param("t_out", lower_constraints=["t_anc"])
no_pulse_model.move_lineages("S1", "Outgroup", t="t_out")
add_pulse_model = no_pulse_model.copy()
add_pulse_model.add_pulse_param("p_pulse")
add_pulse_model.move_lineages(
    "Adm", "GhostS1", t=4.5e4, p="p_pulse")
add_pulse_model.add_time_param(
    "t_ghost", lower=5e4,
    upper_constraints=["t_anc"])
add_pulse_model.move_lineages(
    "GhostS1", "S1", t="t_ghost")
add_pulse_model = no_pulse_model.copy()
add_pulse_model.add_pulse_param("p_pulse")
add_pulse_model.add_time_param(
    "t_pulse", upper_constraints=["t_Adm_S2"])
add_pulse_model.move_lineages(
    "Adm", "GhostS1", t="t_pulse", p="p_pulse")
add_pulse_model.add_time_param(
    "t_ghost", lower_constraints=["t_pulse"], 
    upper_constraints=["t_anc"])
add_pulse_model.move_lineages(
    "GhostS1", "S1", t="t_ghost")

In [29]:
results = []
n_runs = 1
for i in range(n_runs):
    print(f"Starting run {i+1} out of {n_runs}...")
    add_pulse_model.set_params(
        # parameters inherited from no_pulse_model are set to their previous values
        no_pulse_model.get_params(),
        # other parmaeters are set to random initial values
        randomize=True)

    results.append(add_pulse_model.optimize(options={"maxiter":200}))

# sort results according to log likelihood, pick the best (largest) one
best_result = sorted(results, key=lambda r: r.log_likelihood)[-1]

add_pulse_model.set_params(best_result.parameters)
best_result

Starting run 1 out of 1...


            fun: 0.47072173652488153
            jac: array([ 4.90444393e-12, -1.08352523e-13,  2.90258417e-12, -2.27054213e-06,
       -1.98129404e-06,  5.16489738e-07])
  kl_divergence: 0.47072173652488153
 log_likelihood: -86132.33641706174
        message: 'Converged (|f_n-f_(n-1)| ~= 0)'
           nfev: 70
            nit: 14
     parameters: ParamsDict({'t_Adm_S2': 332430.3735197506, 't_anc': 853161.1337360863, 't_out': 2392914.1137825176, 'p_pulse': 0.36526641639511886, 't_pulse': 331770.053789823, 't_ghost': 331808.0934840339})
         status: 1
        success: True
              x: array([ 3.22430374e+05,  5.20730760e+05,  1.53975298e+06, -5.52578362e-01,
        6.21947324e+00, -9.52555251e+00])

In [38]:
pars = best_result.parameters
os.remove('output/momi_rep{}.txt'.format(replicate_id))
with open('output/momi_rep{}.txt'.format(replicate_id), 'a') as output:
    for p in pars:
        output.write('{0}\t{1}\n'.format(p, str(float(pars[p]))))